In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import statsmodels.formula.api as smf
import statsmodels.api as sm

# read

In [2]:
TRAIN_PATH = "/workspace/data/titanic/train.csv"
TEST_PATH = "/workspace/data/titanic/test.csv"

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# EDA後の変更

## 以下のカラムはモデリングに使わない

In [3]:
drop_columns = ['Name', 'Ticket', 'Cabin', 'Embarked']

train_df = train_df.drop(drop_columns, axis=1)
test_df = test_df.drop(drop_columns, axis=1)

## カテゴリ変数を数値変数に変更

In [4]:
# カテゴリデータ一のリスト
type_df = pd.DataFrame(train_df.dtypes.reset_index())
type_df = type_df.rename(columns={0:"type"})
type_df['type'] = type_df['type'].apply(lambda x: str(x))
str_columns = list(type_df.query("type in 'object'")["index"].unique())

# label encoder
for c in str_columns:
    le = LabelEncoder()
    le.fit(train_df[c])
    train_df[c] = le.transform(train_df[c])
    test_df[c] = le.transform(test_df[c])

## AgeのNULL埋めを「Pclass × Sex」による平均年齢

In [5]:
train_list = []
test_list = []
for i, pclass in enumerate(sorted(train_df['Pclass'].unique())):
    for n, sex in enumerate(sorted(train_df['Sex'].unique())):
        
        _df = train_df.query("(Pclass == @pclass) & (Sex == @sex)").copy()
        _df = _df.fillna({"Age":_df['Age'].mean()})
        train_list.append(_df)
        
        _df = test_df.query("(Pclass == @pclass) & (Sex == @sex)").copy()
        _df = _df.fillna({"Age":_df['Age'].mean()})
        test_list.append(_df)
        
train_df = pd.concat(train_list)
test_df = pd.concat(test_list)

train_df = train_df.sort_values("PassengerId")
test_df = test_df.sort_values("PassengerId")

## 標準化
※ 正則化にL2ノルムを使用するため

In [6]:
num_cols = list(train_df.drop(["PassengerId", "Survived"], axis=1).columns)

scaler = StandardScaler()
scaler.fit(train_df[num_cols])

train_df[num_cols] = scaler.transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

## testデータのNULL平均埋め

In [31]:
test_df = test_df.fillna(test_df.mean())

# 学習

In [32]:
train_x = train_df.drop(["PassengerId", "Survived"], axis=1)
train_y = train_df[["Survived"]]
test_x = test_df.drop("PassengerId", axis=1)

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [33]:
columns = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]
svm = LinearSVC(C=0.1)
svm.fit(tr_x, tr_y)
print(svm.score(tr_x, tr_y))

0.8023952095808383


/usr/local/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [43]:
pred = pd.DataFrame({"Survived":svm.predict(test_x)})

# Idを合わせる
pred = pred.reset_index()
pred = pred.rename(columns={'index':'PassengerId'})
pred['PassengerId'] = pred['PassengerId'].apply(lambda x: x + 892)

In [44]:
path = "/workspace/data/titanic/result_svm.csv"
pred.to_csv(path, index=False, encoding='utf-8-sig')